# CS 6780: Advanced Machine Learning

### <i>Enhancing Pairs Trading: The Power of Unsupervised Learning Approaches</i>

## Feature Generation

In this notebook, we create our feature vectors for our unsupervised learning clustering methods. Our features contain at a high level two key pieces of information: (i) the returns of the stock on a monthly basis over the last 72 months and (ii) the firm characteristics of the stock on a quarterly basis over the last 72 months. In so doing, we capture both quantitative and qualitative aspects of the stock. 

### 1. Returns of the Stock Dataset: Filtering, Cleaning, and Generating

In this section, we load, clean, and filter our data for part (i) of our feature vector. We obtain our data from the Center for Research in Security Prices (CRSP), focusing on stocks with common shares listed on the New York Stock Exchange (NYSE), American Stock Exchange (AMEX), and Nasdaq. We omit stocks that have been delisted from these exchanges, as we cannot trade them, as well as stocks missing 25% of their data to ensure the robustness of our analysis. We also exclude stocks with low trading volumes due to their inherent illiquidity and potential for heightened volatility. Our sampling period is 1/2010 to 1/2016.

In [527]:
import pandas as pd

df = pd.read_csv('/Users/kevinwon/Desktop/quant/data.csv')

# Convert "date" column to a datetime object for later use
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')

df.head()

,PERMNO,date,SHRCD,EXCHCD,TICKER,DLSTCD,PRC,VOL,RET,SHROUT
0,10001,2010-01-29,11,2,EGAS,NaN,10.0600,3104.0,-0.018932,4361.0
1,10001,2010-02-26,11,2,EGAS,NaN,10.0084,1510.0,-0.000656,4361.0
2,10001,2010-03-31,11,2,EGAS,NaN,10.1700,2283.0,0.020643,4361.0
3,10001,2010-04-30,11,2,EGAS,NaN,11.3900,3350.0,0.124385,6070.0
4,10001,2010-05-28,11,2,EGAS,NaN,11.4000,3451.0,0.004829,6071.0


In [528]:
print(f"Number of rows in dataset: {len(df)}", )
print(f"Number of stocks in dataset: {len(set(df['PERMNO']))}")

Number of rows in dataset: 274427
Number of stocks in dataset: 5339


#### Exclusion of Delisted Stocks

Below are the delisting codes for CRSP:

- (1) Still trading or halted but not yet delisted
- (2) Merger
- (3) Exchange
- (4) Liquidation
- (5) Delisted by NYSE, AMEX, or Nasdaq
- (7) Delisted by the Securities and Exchange Commission (SEC)
- (8) Trading simultaneously on more than one exchange

We remove stocks with a delisting code of 

- (2) Because stocks in the midst of a merger may exhibit altered trading dynamics
- (3) Because a stock's transfer to a different exchange could potentially impact its volatility and liquidity
- (4) Because the process of liquidation frequently precedes delisting and can significantly influence the stock's market value
- (5) Because delisting from prominent exchanges such as the NYSE, AMEX, or Nasdaq signifies significant underlying financial or operational issues
- (7) Because delisting by the SEC usually occurs due to severe violations of regulatory standards or failure to adhere to financial reporting requirements

In [529]:
# Convert 'DLSTCD' to a string and pad with zeros to ensure 3 digits
df['DLSTCD_str'] = df['DLSTCD'].astype(str).str.pad(3, fillchar='0')

# Filter the DataFrame based on the condition that the hundredth digit is 2, 3, 4, 5, or 7
delisted_stocks = df[df['DLSTCD_str'].str[0].isin(['2', '3', '4', '5', '7'])]

# Extract the PERMNO identifiers for these filtered rows
permno_list_to_remove_delist = list(set(delisted_stocks['PERMNO']))

#### Exclusion of Illiquid Stocks

We add a "turnover" column to our dataset, defined as trading volume divided by shares outstanding, which serves as a key indicator of liquidity. To establish a robust liquidity threshold for the exclusion of low-turnover stocks, we conduct various statistical tests on the turnover data. We calculate percentiles, mean, median, and standard deviation to determine an appropriate threshold. We decide to exclude stocks that fall below the 25th percentile, as this threshold strikes an optimal balance, offering the potential for higher returns due to the wider bid-ask spreads characteristic of less liquid stocks, while also acknowledging the associated increased risks.

In [530]:
# Add a "turnover" column to the dataset
df['Turnover'] = df['VOL']/df['SHROUT']
df.head()

,PERMNO,date,SHRCD,EXCHCD,TICKER,DLSTCD,PRC,VOL,RET,SHROUT,DLSTCD_str,Turnover
0,10001,2010-01-29,11,2,EGAS,NaN,10.0600,3104.0,-0.018932,4361.0,nan,0.711763
1,10001,2010-02-26,11,2,EGAS,NaN,10.0084,1510.0,-0.000656,4361.0,nan,0.346251
2,10001,2010-03-31,11,2,EGAS,NaN,10.1700,2283.0,0.020643,4361.0,nan,0.523504
3,10001,2010-04-30,11,2,EGAS,NaN,11.3900,3350.0,0.124385,6070.0,nan,0.551895
4,10001,2010-05-28,11,2,EGAS,NaN,11.4000,3451.0,0.004829,6071.0,nan,0.568440


In [531]:
# Basic statistics
mean_turnover = df['Turnover'].mean()
median_turnover = df['Turnover'].median()
std_dev_turnover = df['Turnover'].std()

print(f"Mean Turnover: {mean_turnover}")
print(f"Median Turnover: {median_turnover}")
print(f"Standard Deviation of Turnover: {std_dev_turnover}")

# Percentile analysis
percentiles = [10, 25, 50, 75, 90]
percentile_values = df['Turnover'].quantile([p / 100 for p in percentiles]).to_dict()

print("\nTurnover Percentiles:")
for percentile, value in percentile_values.items():
    print(f"{percentile * 100}th percentile: {value}")

# Exclude stocks below the 25th percentile
threshold = percentile_values[0.25]
print(f"\nSuggested Threshold (25th percentile): {threshold}")

illiquid_stocks = df[df['Turnover'] <= threshold] 
permno_list_to_remove_illiquid = list(set(illiquid_stocks['PERMNO']))

Mean Turnover: 1.75026094145777
Median Turnover: 1.1256161859933522
Standard Deviation of Turnover: 3.62242264675365

Turnover Percentiles:
10.0th percentile: 0.1787505648026872
25.0th percentile: 0.4925106505213936
50.0th percentile: 1.1256161859933522
75.0th percentile: 2.089508284144307
90.0th percentile: 3.610888920987581

Suggested Threshold (25th percentile): 0.4925106505213936


However, we also want to ensure that only stocks with a significant history of low turnover are considered, reducing the chance of excluding stocks due to short-term anomalies or data issues. If the number of months the stock has low turnover is at maximum 5, we do not remove it from our dataset

In [532]:
for permno in permno_list_to_remove_illiquid:
    temp_df = illiquid_stocks[illiquid_stocks['PERMNO'] == permno]
    if len(temp_df) < 5:     
        permno_list_to_remove_illiquid.remove(permno)

We now remove all stocks from the dataset based on the above conditions. Note, we haven't removed any stocks yet because of missing values in the dataset; we will handle that step later once we generate a pivot table for the stocks.

In [533]:
final_permno_list_to_remove = list(set(permno_list_to_remove_delist).union(set(permno_list_to_remove_illiquid)))
filtered_df = df[df['PERMNO'].isin(final_permno_list_to_remove) == False]

print(f"Dataset size: {len(filtered_df)}")
print(f"Number of stocks: {len(set(filtered_df['PERMNO']))}")
filtered_df.head()

Dataset size: 123664
Number of stocks: 2119


,PERMNO,date,SHRCD,EXCHCD,TICKER,DLSTCD,PRC,VOL,RET,SHROUT,DLSTCD_str,Turnover
110,10025,2010-01-29,11,3,AEPI,NaN,34.91,8653.0,-0.088036,6849.0,nan,1.263396
111,10025,2010-02-26,11,3,AEPI,NaN,34.90,10045.0,-0.000286,6850.0,nan,1.466423
112,10025,2010-03-31,11,3,AEPI,NaN,26.02,20299.0,-0.254441,6850.0,nan,2.963358
113,10025,2010-04-30,11,3,AEPI,NaN,27.63,14573.0,0.061875,6850.0,nan,2.127445
114,10025,2010-05-28,11,3,AEPI,NaN,25.14,11993.0,-0.090119,6850.0,nan,1.750803


#### Pivot Table Creation & Exclusion of Stocks Missing Data

Let's now create a pivot table for our stocks and handle the missing data problem. Here is how we do this. We restructure our dataset into a pivot table format, where each row corresponds to a specific stock (identified by 'PERMNO'), and each column represents a different date. In this arrangement, the cell values at the intersection of each stock row and date column indicate the stock's return ('RET') on that particular date. We then clean the data by removing stocks with over 25% missing values, ensuring a robust dataset. Remaining missing values are filled using a forward-fill method limited to 5 consecutive fills, balancing data integrity with practical imputation. 

In [534]:
# Convert 'RET' to numeric, coercing errors to NaN
filtered_df = filtered_df.copy()
filtered_df['RET'] = pd.to_numeric(filtered_df['RET'], errors='coerce')

# Get the pivot table, with stock permno as index and datetime as columns
stocks = pd.pivot_table(filtered_df,values='RET',index='PERMNO',columns='date')

# Delete stocks that don't have at least 75% of its values as non-missing
stocks = stocks.dropna(thresh = 3*len(stocks.columns)//4)

# Fills in some of the remaining missing values in each row, using the most recent non-missing value
stocks = stocks = stocks.ffill(axis=1, limit=5)

stocks.head()

date,2010-01-29,2010-02-26,2010-03-31,2010-04-30,2010-05-28,2010-06-30,2010-07-30,2010-08-31,2010-09-30,2010-10-29,...,2015-03-31,2015-04-30,2015-05-29,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-30,2015-11-30,2015-12-31
PERMNO,,,,,,,,,,,,,,,,,,,,,
10025,-0.088036,-0.000286,-0.254441,0.061875,-0.090119,-0.050119,0.210218,-0.181661,-0.001268,0.031753,...,0.086459,-0.089753,-0.001397,0.103338,-0.124094,0.126163,0.052893,0.395430,0.139750,-0.153871
10032,0.194171,0.014114,0.044651,0.029142,-0.081715,-0.214684,0.091997,-0.211901,0.275394,0.034072,...,0.012919,0.055923,0.056446,-0.035180,-0.130811,-0.001835,0.013396,-0.102644,0.074523,-0.061290
10051,0.175705,0.146986,-0.025201,0.025302,-0.085300,0.053372,-0.045100,-0.239067,0.114176,0.287483,...,-0.123600,-0.015425,0.029096,0.019574,-0.076792,-0.171442,-0.239264,0.057185,0.079750,0.056519
10104,-0.057888,0.068951,0.043002,0.008071,-0.127477,-0.049180,0.103914,-0.075931,0.229114,0.096089,...,-0.015290,0.014368,-0.002980,-0.073350,-0.005211,-0.071357,-0.026153,0.079457,0.003347,-0.062612
10107,-0.075459,0.022001,0.021538,0.042595,-0.150811,-0.108140,0.121686,-0.085819,0.043682,0.088812,...,-0.072862,0.196409,-0.030222,-0.057832,0.057758,-0.061456,0.017004,0.189336,0.039324,0.020791


Let's now check the number of stocks we have after all the data preprocessing we've done, and let's also check that the average number of non-null values per row is close to 72 to ensure our data imputation worked.

In [535]:
print(f"Number of stocks: {len(stocks)}")
print(f"Average number of non-null values: {stocks.count(axis=1).mean()}")

Number of stocks: 1556
Average number of non-null values: 71.54627249357327


Let's save this stock pivot table as a .csv file and the list of the PERMNO identifiers in the stock pivot table as a .txt file in our directory.

In [536]:
permno = list(stocks.index)
with open('permno_list.txt', 'w') as filehandle:   
    for listitem in permno:
        filehandle.write('%i\n' % listitem)

stocks.to_csv('stock_price_table.csv')